In [1]:
! pip install pyspark


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:

from pyspark.sql.types import *
from pyspark.sql.window import Window

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .config("hive.metastore.uris", "thrift://172.27.1.8:9083")\
      .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
      .enableHiveSupport() \
      .getOrCreate() 

24/04/04 15:50:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
spark.sql("select * from importaciones_db.marca").show(truncate=False)

+-------+--------------+
|idmarca|nombremarca   |
+-------+--------------+
|1      |ACURA         |
|2      |AUDI          |
|6      |BAJAJ         |
|7      |BLUE BIRD     |
|4      |BMW           |
|5      |BOMBARDIER LTD|
|3      |BUICK         |
|8      |CADILLAC      |
|9      |CHEVROLET     |
|10     |CHRYSLER      |
|11     |CITROEN       |
|12     |COMMER        |
|15     |DAEWOO        |
|13     |DODGE         |
|16     |DONG FENG     |
|17     |DORSEY        |
|14     |DUCATI        |
|21     |FERRARI       |
|19     |FIAT          |
|18     |FORD          |
+-------+--------------+
only showing top 20 rows



In [11]:
spark.sql("select * from importaciones_db.linea").show(truncate=False)

+-------+----------------------+
|idlinea|nombrelinea           |
+-------+----------------------+
|1      |TL                    |
|2      |RSX                   |
|3      |RSX TYPE-S            |
|4      |INTEGRA GS            |
|5      |TSX                   |
|6      |RSX  S                |
|7      |RDX AWD               |
|8      |CSX                   |
|9      |MDX TOURING 4X4       |
|10     |RSX TYPE S            |
|11     |A4 2.0T QUATTRO       |
|12     |A4 1.8T QUATTRO       |
|13     |A4 QUATTRO            |
|14     |A4 2.0T               |
|15     |Q7 QUATTRO PREMIUM AWD|
|16     |A4                    |
|17     |Q5 PRESTIGE 4X4       |
|18     |Q7 4.2 QUATTRO 4X4    |
|19     |A3                    |
|20     |A6 3.2 QUATTRO        |
+-------+----------------------+
only showing top 20 rows



In [14]:
spark.sql("select * from importaciones_db.TIPO_COMBUSTIBLE").show(truncate=False)

+-----------------+---------------------+
|idtipocombustible|nombretipocombustible|
+-----------------+---------------------+
|1                |GASOLINA             |
|2                |DIESEL               |
|3                |OTROS                |
+-----------------+---------------------+



In [17]:
spark.sql("select * from importaciones_db.TIPO_IMPORTADOR").show(truncate=False)

+----------------+--------------------+
|idtipoimportador|nombretipoimportador|
+----------------+--------------------+
|1               |OCASIONAL           |
|2               |DISTRIBUIDOR        |
+----------------+--------------------+



In [16]:
spark.sql("select * from importaciones_db.TIPO_VEHICULO").show(truncate=False)

+--------------+------------------+
|idtipovehiculo|nombretipovehiculo|
+--------------+------------------+
|1             |AUTOMOVIL         |
|2             |CAMIONETA         |
|3             |MOTO              |
|4             |CUATRIMOTO        |
|5             |TRIMOTO           |
|6             |BUS               |
|7             |PICK UP           |
|8             |CAMION            |
|9             |CAMION FURGON     |
|10            |MICROBUS          |
|11            |JEEP              |
|12            |CAMIONETILLA      |
|13            |PANEL             |
|14            |REMOLQUE          |
|15            |CAMION BOMBEROS   |
|16            |PORTA CONTENEDOR  |
|17            |PLATAFORMA        |
|18            |AUTOBUS           |
|19            |CAMION GRUA       |
|20            |AMBULANCIA        |
+--------------+------------------+
only showing top 20 rows



In [18]:
spark.sql("select * from importaciones_db.PAIS_ORIGEN").show(truncate=False)

+------------+---------------------------------------+
|idpaisorigen|nombrepaisorigen                       |
+------------+---------------------------------------+
|1           |ESTADOS UNIDOS                         |
|2           |JAPON                                  |
|3           |CANADA                                 |
|4           |ALEMANIA REP. FED.                     |
|5           |SLOVAKIA                               |
|6           |ESPANA                                 |
|7           |INDIA                                  |
|8           |CHINA                                  |
|9           |MEXICO                                 |
|10          |COREA DEL SUR                          |
|11          |KOREA DEL NORTE (REPUBLICA DEMOCRATICA)|
|12          |FRANCIA                                |
|13          |ITALIA                                 |
|14          |TURQUIA                                |
|15          |TAIWAN                                 |
|16       

In [40]:
spark.sql("select * from importaciones_db.IMPORTACION").show(truncate=False)

+-------------+---------------+--------------+----------------+-------------------+------------------+----------------+--------+--------+-------+--------+--------+
|idimportacion|idpais_idaduana|idlinea_modelo|idtipovehiculofk|idtipocombustiblefk|idtipoimportadorfk|fechaimportacion|valorcif|impuesto|puertas|tonelaje|asientos|
+-------------+---------------+--------------+----------------+-------------------+------------------+----------------+--------+--------+-------+--------+--------+
|1            |1              |1             |1               |1                  |1                 |17/01/2017      |11926.16|1431.14 |4      |0.0     |5       |
|2            |2              |2             |1               |1                  |1                 |23/01/2017      |21652.99|2598.36 |4      |0.0     |5       |
|3            |3              |3             |1               |1                  |1                 |19/01/2017      |9928.89 |1191.47 |2      |0.0     |5       |
|4            |4

In [41]:
spark.sql("select * from importaciones_db.pais_aduana").show(truncate=False)

+---------------+------------+---------------+
|idpais_idaduana|idpaisorigen|idaduanaingreso|
+---------------+------------+---------------+
|1              |1           |1              |
|2              |1           |2              |
|17             |1           |3              |
|16             |1           |4              |
|18             |1           |5              |
|23             |1           |6              |
|19             |1           |7              |
|32             |1           |8              |
|3              |2           |1              |
|4              |2           |2              |
|5              |2           |3              |
|6              |3           |1              |
|33             |3           |2              |
|24             |3           |3              |
|7              |4           |1              |
|9              |4           |2              |
|14             |4           |3              |
|8              |4           |4              |
|10          

In [42]:
spark.sql("select * from importaciones_db.pais_origen").show(truncate=False)

+------------+---------------------------------------+
|idpaisorigen|nombrepaisorigen                       |
+------------+---------------------------------------+
|1           |ESTADOS UNIDOS                         |
|2           |JAPON                                  |
|3           |CANADA                                 |
|4           |ALEMANIA REP. FED.                     |
|5           |SLOVAKIA                               |
|6           |ESPANA                                 |
|7           |INDIA                                  |
|8           |CHINA                                  |
|9           |MEXICO                                 |
|10          |COREA DEL SUR                          |
|11          |KOREA DEL NORTE (REPUBLICA DEMOCRATICA)|
|12          |FRANCIA                                |
|13          |ITALIA                                 |
|14          |TURQUIA                                |
|15          |TAIWAN                                 |
|16       

### Enunciado 1
¿Cuál es el total de valor del Costo, Seguro y Flete (CIF) de las importaciones por país en un año específico, , ordenado por el total de CIF en orden descendente?

In [60]:
spark.sql("""select
tc.nombrepaisorigen, sum(valorcif) valor_cif
from importaciones_db.importacion ta
	left join importaciones_db.pais_aduana tb on ta.IdPais_IdAduana = tb.idpais_idaduana
	left join importaciones_db.pais_origen tc on tb.IdPaisOrigen = tc.idpaisorigen
where substr(FechaImportacion, 7, 4)  = 2017
group by tc.nombrepaisorigen
order by 2 desc;""").show()

+------------------+--------------------+
|  nombrepaisorigen|           valor_cif|
+------------------+--------------------+
|             CHINA| 4.774244300560189E9|
|             INDIA| 1.570532227949962E9|
|             JAPON|  3.52988131369999E8|
|         TAILANDIA| 2.709255181200002E8|
|     COREA DEL SUR| 1.853637509100004E8|
|            BRASIL|      1.1451495324E8|
|    ESTADOS UNIDOS| 8.642539742000003E7|
|            CANADA| 2.380309717999999E7|
|            MEXICO|1.9315851560000002E7|
|ALEMANIA REP. FED.|   9254979.390000006|
|         INDONESIA|   7981075.059999999|
|       REINO UNIDO|  6127052.0600000005|
|         ARGENTINA|  1990308.9500000002|
|           BELGICA|  1476442.0799999998|
|          SLOVAKIA|          1276098.13|
|          COLOMBIA|   932414.8799999999|
|           FRANCIA|           704772.75|
|            SUECIA|  434876.72000000003|
|            ESPANA|           181283.12|
|           AUSTRIA|           180921.07|
+------------------+--------------

### Enunciado 2
¿Cuál es el valor total y el valor promedio de impuestos recaudados por años de las importaciones vehiculares?

In [45]:
spark.sql("""select
	substr(ta.FechaImportacion, 7, 4) as aio,
	sum(ta.impuesto) total,
	round(avg(ta.impuesto), 3) promedio
from importaciones_db.importacion ta
group by substr(ta.FechaImportacion, 7, 4)
order by 1 desc""").show()

+----+-------------------+---------+
| aio|              total| promedio|
+----+-------------------+---------+
|2017|8.917512902700138E8|49921.698|
+----+-------------------+---------+



### Enunciado 3
¿Cuál es el total de impuestos recaudados por tipo de importador en un año específico?


In [64]:
spark.sql("""
select
	tb.nombretipoimportador,
	sum(impuesto) total
from importaciones_db.importacion ta
	left join importaciones_db.tipo_importador tb on ta.idtipoimportadorfk = tb.idtipoimportador
where substr(FechaImportacion, 7, 4)  = 2017
group by tb.nombretipoimportador
order by 1
""").show()

+--------------------+-------------------+
|nombretipoimportador|              total|
+--------------------+-------------------+
|        DISTRIBUIDOR|1.206381834599995E8|
|           OCASIONAL|7.711131068099397E8|
+--------------------+-------------------+



### Enunciado 4
¿Cuál es la aduana que ha recaudado más ingresos en un año determinado, ordenado por el total de ingresos en orden descendente?

In [69]:
spark.sql("""
select tc.NombreAduanaIngreso, sum(valorcif) valor_cif
from importaciones_db.importacion ta
	left join importaciones_db.pais_aduana tb on ta.IdPais_IdAduana = tb.IdPais_IdAduana
	left join importaciones_db.aduana_ingreso tc on tb.IdAduanaIngreso = tc.IdAduanaIngreso
where substr(FechaImportacion, 7, 4)  = 2017
group by tc.NombreAduanaIngreso
order by 2 desc;
""").show()

24/04/04 17:08:59 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
24/04/04 17:08:59 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
24/04/04 17:08:59 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.


+--------------------+--------------------+
| NombreAduanaIngreso|           valor_cif|
+--------------------+--------------------+
|      PUERTO QUETZAL| 7.077334003959732E9|
|SANTO TOMAS DE CA...|1.4854394534999973E8|
|           EL CARMEN|6.5388829570000045E7|
|      PUERTO BARRIOS| 6.191948598000011E7|
|          TECUN UMAN| 5.912543075999993E7|
|       EXPRESS AEREO|   7425997.800000004|
|   PEDRO DE ALVARADO|  2777197.9800000004|
|                  G8|  2574271.4599999995|
|                  G4|  1837049.7099999997|
|                  G1|          1508663.76|
|   MELCHOR DE MENCOS|  206389.71999999997|
|         VALLE NUEVO|           113095.83|
|       SAN CRISTOBAL|           101742.68|
|            EL CEIBO|            50403.34|
|          LA MESILLA|            23685.01|
|       AGUA CALIENTE|            22665.54|
+--------------------+--------------------+



### Enunciado 5
¿Cuáles son las 5 aduanas con el mayor número de importaciones de BUSES en términos de cantidad de vehículos?

In [70]:
spark.sql("""
select tc.NombreAduanaIngreso, count(*) valor_cif
from importaciones_db.importacion ta
	left join importaciones_db.pais_aduana tb on ta.IdPais_IdAduana = tb.IdPais_IdAduana
	left join importaciones_db.aduana_ingreso tc on tb.IdAduanaIngreso = tc.IdAduanaIngreso
where substr(FechaImportacion, 7, 4)  = 2017
group by tc.NombreAduanaIngreso
order by 2 desc
limit 5
""").show()

24/04/04 17:14:23 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
24/04/04 17:14:23 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
24/04/04 17:14:23 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.


+--------------------+---------+
| NombreAduanaIngreso|valor_cif|
+--------------------+---------+
|      PUERTO QUETZAL|     8142|
|      PUERTO BARRIOS|     3358|
|           EL CARMEN|     2985|
|          TECUN UMAN|     1745|
|SANTO TOMAS DE CA...|     1188|
+--------------------+---------+



### Enunciado 6
¿Cuál es el total de impuestos registrados por año y cuánto fue en el año siguiente?


In [74]:
spark.sql("""
select
	substr(FechaImportacion, 7, 4) as periodo,
	sum(impuesto) total,
	LEAD(SUM(impuesto), 1) OVER (ORDER BY substr(FechaImportacion, 7, 4)) AS importacion_siguiente_anio
from importaciones_db.importacion ta
group by substr(FechaImportacion, 7, 4)
order by 2 desc;
""").show()


24/04/04 17:18:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/04 17:18:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/04 17:18:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/04 17:18:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/04 17:18:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/04 17:18:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/04 1

+-------+-------------------+--------------------------+
|periodo|              total|importacion_siguiente_anio|
+-------+-------------------+--------------------------+
|   2017|8.917512902700138E8|                      NULL|
+-------+-------------------+--------------------------+

